In [ ]:
# Author: Riley Owens (GitHub: mrileyowens)

# This file measures values and errors of 
# various Lyα profile parameters.

In [ ]:
import sys

import numpy as np
np.set_printoptions(threshold=sys.maxsize)

import pandas as pd

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi']=100
plt.rcParams['lines.linewidth']=0.5
from matplotlib.offsetbox import AnchoredText

from scipy.optimize import curve_fit
from scipy.interpolate import interp1d

In [ ]:
def gaussFit(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

def fitNormPeak(vArr,fArr,sampleMin,sampleMax,params,bounds):
    
    fArr=fArr[vArr > sampleMin]
    vArr=vArr[vArr > sampleMin]
    fArr=fArr[vArr < sampleMax]
    vArr=vArr[vArr < sampleMax]
    
    popt,pcov=curve_fit(gaussFit,vArr,fArr,p0=params,bounds=bounds)
    
    return popt,pcov

def lin_interp(x, y, i, half):
    return x[i] + (x[i+1] - x[i]) * ((half - y[i]) / (y[i+1] - y[i]))

def half_max_x(x, y, Min, Max,ctr):
    
    y=np.where(x>Min,y,0.0)
    y=np.where(x<Max,y,0.0)
    
    f=interp1d(x,y)
    peak=f(ctr)
    
    half = peak/2.0
    signs = np.sign(np.add(y, -half))
    zero_crossings = (signs[0:-2] != signs[1:-1])
    zero_crossings_i = np.where(zero_crossings)[0]
    
    return [lin_interp(x, y, zero_crossings_i[0], half),
            lin_interp(x, y, zero_crossings_i[1], half)],half

#def v2w(v,redshift):
    
#    w=1215.67*(1.0+redshift)*(1.0+(v/3e5))
    
#    return w

def cont(wArr,fArr,Min,Max):
    
    fArr=fArr[wArr > Min]
    wArr=wArr[wArr > Min]
    fArr=fArr[wArr < Max]
    wArr=wArr[wArr < Max]
    
    cont=np.mean(fArr)
    
    return cont

#def bounds(x,y,cont,b1,b2,m,n):
    
    #y=np.where(x>=b2,y,0.0)
    #y=np.where(x<=b1,y,0.0)
    
    #print(y)
    
#    y=y[x > b1]
#    x=x[x > b1]
#    y=y[x < b2]
#    x=x[x < b2]
    
#    signs=np.sign(np.add(y, -cont))
#    zero_crossings=(signs[0:-2] != signs[1:-1])
#    zero_crossings_i=np.where(zero_crossings)[0]
    
#    return [lin_interp(x, y, zero_crossings_i[m], cont),
            lin_interp(x, y, zero_crossings_i[n], cont)]

def smooth(y, box_pts):

    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    
    return y_smooth

def peaks(x,y,x0,sigma,slit):
    
    y=y[(x >= -1000.0) & (x <= 1000.0)]
    x=x[(x >= -1000.0) & (x <= 1000.0)]
    
    signs=np.sign(y)
    zero_crossings=(signs[0:-2] != signs[1:-1])
    zero_crossings_i=np.where(zero_crossings)[0]
    
    intp=np.array([])
    
    for i in zero_crossings_i:
        
        n=lin_interp(x,y,i,0.0)
        
        intp=np.append(intp,n)
    
    if slit=='H6':
        
        bP=np.mean(intp[(intp >= -200.0) & (intp <= 0.0)])
        rP=np.mean(intp[(intp >= x0+1.5*sigma) & (intp <= 320.0)])
        print(intp)
        
    elif slit=='H3':
        
        bP=np.mean(intp[(intp >= -200.0) & (intp <= -10.0)])
        rP=np.mean(intp[(intp >= 200.0) & (intp <= 320.0)])
    
    else:
    
        bP=np.mean(intp[(intp >= -200.0) & (intp <= x0-1.5*sigma)])
        rP=np.mean(intp[(intp >= x0+1.5*sigma) & (intp <= 350.0)])
    
    return bP,rP,intp

def ewfluxlum(wArr,fArr,cont,z):
    
    Min=1212.0*(z+1.0)
    Max=1221.0*(z+1.0)
    
    fArr=fArr[wArr > Min]
    wArr=wArr[wArr > Min]
    fArr=fArr[wArr < Max]
    wArr=wArr[wArr < Max]
    
    flux=np.trapz(fArr-cont,wArr)
    lum=4*np.pi*flux*((((2.998e10)*z)/(2.26830849e-18))**2)
    ew=abs(np.trapz(1.0-np.divide(fArr,cont),wArr))
    
    
    return ew,flux,lum

In [ ]:
# Establishing directories and filepaths
home='C://Users/15136/OneDrive - University of Cincinnati/Documents/Research/sunburstarc'
data=home+'/data/spectra/mage'
figs=home+'/figs'

In [ ]:
# Declaring plot labels
wlabel='Wavelength (Å)'
vlabel='Velocity (km s$^{-1}$)'
flabel='Flux (10$^{-16}$ erg s$^{-1}$ cm$^{-2}$ Å$^{-1}$)'

In [ ]:
df=pd.read_csv(data+'/slits.txt',delimiter=' ',header=0)

# String array containing slit names
slits=df.iloc[:,0].to_numpy(dtype=str)

# String array containing MAGE spectra filepaths
files=df.iloc[:,1].to_numpy(dtype=str)

# String array containing FIRE spectra filepaths
fire=df.iloc[:,17].to_numpy(dtype=str)

# Array containing redshifts of each slit
z=df.iloc[:,2].to_numpy(dtype=np.float64)

# Initial guesses for central peak fit parameters
ampGuess=df.iloc[:,3].to_numpy()
ctrGuess=df.iloc[:,4].to_numpy()
stdGuess=df.iloc[:,5].to_numpy()

Min=df.iloc[:,6].to_numpy(dtype=np.float64)
Max=df.iloc[:,7].to_numpy(dtype=np.float64)

# Upper / lower bounds of central peak fit parameters
ctrLBnd=df.iloc[:,8].to_numpy(dtype=np.float64)
stdLBnd=df.iloc[:,9].to_numpy(dtype=np.float64)
ctrUBnd=df.iloc[:,10].to_numpy(dtype=np.float64)
stdUBnd=df.iloc[:,11].to_numpy(dtype=np.float64)
contLBnd=df.iloc[:,12].to_numpy(dtype=np.float64)
contUBnd=df.iloc[:,13].to_numpy(dtype=np.float64)

# Boolean arrays which indicate if clump has central
# or blue peaks in Lyα profile
cPeak=df.iloc[:,14].to_numpy(dtype=bool)
bPeak=df.iloc[:,15].to_numpy(dtype=bool)

# Boolean array which indicates if clump leaks LyC
lyc=df.iloc[:,16].to_numpy(dtype=bool)

In [ ]:
# Initializing figure of histograms of various line measurements
# after noise perturbation
figErr,axErr=plt.subplots(8,6,figsize=(18,24),sharey='row')

# Running loop across each slit
for i in np.indices(np.shape(slits)).flatten():
    
    #Converting .txt file with MAGE data into a data frame
    dataFrame=pd.read_csv(data+'/'+files[i],delimiter='\t',header=0,skiprows=13)

    #Discarding missing data
    dataFrame=dataFrame.apply(pd.to_numeric,errors='coerce').dropna()

    #Discarding extreme outliers
    dataFrame=dataFrame[dataFrame.iloc[:,1]<1e-20]

    #Resetting the index
    dataFrame=dataFrame.reset_index(drop=True)

    # Extracting wavelength, flux, and noise
    wArr=dataFrame.iloc[:,0].to_numpy()
    fArr=(dataFrame.iloc[:,1].to_numpy())*((2.998e18)/(np.square(wArr)))
    nArr=(dataFrame.iloc[:,2].to_numpy())*((2.998e18)/(np.square(wArr)))

    # Converting wavelength to peculiar velocity
    # of Lyman-α line
    vArr=(3e5)*(((wArr/(1+np.float64(z[i])))/1215.67)-1)
    
    # Computing moving-frame Lyα wavelength
    w=1215.67*(z[i]+1)
    
    # Assigning row title
    title='Slit '+slits[i]
    
    # If there is a blue and red peak
    if cPeak[i]==True & bPeak[i]==True:
        
        fitGuess=(ampGuess[i],ctrGuess[i],stdGuess[i])
        sampleRange=(Min[i],Max[i])
        
        # Fitting central peak
        popt,pcov=fitNormPeak(vArr,fArr*1e16,*sampleRange,params=fitGuess,bounds=([0.0,ctrLBnd[i],0.0],[np.inf,ctrUBnd[i],stdUBnd[i]]))
        
        # Computing local continuum
        cntm=cont(wArr,fArr,contLBnd[i],contUBnd[i])
        
        # Computing EW, flux, luminosity of line profile
        e,f,l=ewfluxlum(wArr,fArr,cntm,z[i])
        l=l*1e-42
        
        deriv=np.gradient(smooth(fArr*1e16-gaussFit(vArr,*popt),5),vArr,edge_order=2)
        
        b,r,intp=peaks(vArr,deriv,popt[1],popt[2],slits[i])
        
        if slits[i]=='X':
            print(intp)
        
        hmxB,halfB=half_max_x(vArr,fArr*1e16-gaussFit(vArr,*popt)-cntm*1e16,-750.0,popt[1],b)
        hmxR,halfR=half_max_x(vArr,fArr*1e16-gaussFit(vArr,*popt)-cntm*1e16,popt[1],750.0,r)
        hmxC,halfC=half_max_x(vArr,gaussFit(vArr,*popt)-cntm*1e16,-1000.0,1000.0,popt[1])
        
        fig1,ax1=plt.subplots(2,1,sharex=True)
        
        ax1[0].plot(vArr,deriv)
        ax1[0].axhline(0.0,ls='dashed',c='black')
        ax1[0].axvline(b,ls='dashed',c='blue')
        ax1[0].axvline(r,ls='dashed',c='red')
        
        ax1[0].set_xlim(-1000.0,1000.0)
        
        ax1[0].set_ylabel('Slope')
        ax1[0].set_title('Derivative')
        
        ax1[1].plot(vArr,fArr*1e16-gaussFit(vArr,*popt))
        ax1[1].axvline(b,ls='dashed',c='blue')
        ax1[1].axvline(r,ls='dashed',c='red')
        
        n=0
        fwhmRArr=np.array([])
        fwhmCArr=np.array([])
        fwhmBArr=np.array([])
        psepArr=np.array([])
        ctrBArr=np.array([])
        ctrRArr=np.array([])
        ewArr=np.array([])
        lumArr=np.array([])
        
        # Repeating measurement 1000 times, adding Gaussian noise each time
        while n!=1000:
            
            # Adding Gaussian noise into measurement from 
            # reported noise
            smthg=np.random.normal(fArr,nArr)
        
            # Computing noise-perturbed local continuum
            cntmN=cont(wArr,smthg,contLBnd[i],contUBnd[i])
            
            # Computing noise=perturbed EW, flux, luminosity of line profile
            eN,fN,lN=ewfluxlum(wArr,smthg,cntmN,z[i])
            lN=lN*1e-42
    
            # Fitting noise-perturbed central peak
            popt1,pcov1=fitNormPeak(vArr,smthg*1e16,*sampleRange,params=popt,bounds=([0.0,ctrLBnd[i],0.0],[np.inf,ctrUBnd[i],stdUBnd[i]]))
    
            peak=gaussFit(vArr,*popt1)
            noCPeak=smthg*1e16-gaussFit(vArr,*popt1)
            
            # Computing noise-perturbed numerical derivative of flux
            derivN=np.gradient(smooth(smthg*1e16-gaussFit(vArr,*popt1),5),vArr,edge_order=2)
        
            bN,rN,intpN=peaks(vArr,derivN,popt1[1],popt1[2],slits[i])
    
            psepN=rN-bN
        
            if slits[i]=='X':
        
                f,a=plt.subplots(2,1,sharex=True)
            
                a[0].plot(vArr,derivN,c='black')
                a[0].axvline(bN,ls='dashed',c='blue',label=str(bN))
                a[0].axvline(rN,ls='dashed',c='red',label=str(rN))
                a[0].axhline(0.0,ls='dashed',c='black')
                
                a[0].legend(loc='upper right')
            
                a[1].plot(vArr,smooth(smthg*1e16-gaussFit(vArr,*popt1),5))
                a[1].plot(vArr,smthg*1e16-gaussFit(vArr,*popt1),ls='dashed',c='gray')
                a[1].axvline(bN,ls='dashed',c='blue')
                a[1].axvline(rN,ls='dashed',c='red')
                a[1].axvline(b,ls='dotted',c='blue')
                a[1].axvline(r,ls='dotted',c='red')
            
                a[1].set_xlim(-1000.0,1000.0)
            
                plt.show()
                
                print(bN,rN,psepN)
    
            hmxRN,halfRN=half_max_x(vArr,noCPeak-cntmN*1e16,popt1[1],750.0,rN)
            hmxCN,halfCN=half_max_x(vArr,peak-cntmN*1e16,-1000.0,1000.0,popt1[1])
            hmxBN,halfBN=half_max_x(vArr,noCPeak-cntmN*1e16,-750.0,popt1[1],bN)
    
            fwhmRN=np.array([hmxRN[1]-hmxRN[0]])
            fwhmCN=np.array([hmxCN[1]-hmxCN[0]])
            fwhmBN=np.array([hmxBN[1]-hmxBN[0]])
    
            fwhmRArr=np.append(fwhmRArr,fwhmRN)
            fwhmCArr=np.append(fwhmCArr,fwhmCN)
            fwhmBArr=np.append(fwhmBArr,fwhmBN)
            psepArr=np.append(psepArr,psepN)
            
            ewArr=np.append(ewArr,eN)
            lumArr=np.append(lumArr,lN)
    
            n=n+1
        
        fwhmRArr=fwhmRArr[~np.isnan(fwhmRArr)]
        print(np.shape(fwhmRArr))
        
        # Assigning different color to 
        # LyC-leaking clumps
        if lyc[i]==True:
            clr='red'
        else:
            clr=None

        # Plotting histogram distributions of noise-perturbed profile measurements
        # compared to measured value from 'true' profile
        axErr[i,0].hist((fwhmBArr-np.mean(fwhmBArr)),bins=30,color=clr)
        axErr[i,0].axvline(((hmxB[1]-hmxB[0])-np.mean(fwhmBArr)),ls='dashed',c='black')
        
        axErr[i,1].hist((fwhmCArr-np.mean(fwhmCArr)),bins=30,color=clr)
        axErr[i,1].axvline(((hmxC[1]-hmxC[0])-np.mean(fwhmCArr)),ls='dashed',c='black')
        
        axErr[i,2].hist((fwhmRArr-np.mean(fwhmRArr)),bins=30,color=clr)
        axErr[i,2].axvline(((hmxR[1]-hmxR[0])-np.mean(fwhmRArr)),ls='dashed',c='black')

        axErr[i,3].hist((psepArr-np.mean(psepArr)),bins=30,color=clr)
        axErr[i,3].axvline(((r-b)-np.mean(psepArr)),ls='dashed',c='black')
        
        axErr[i,4].hist((ewArr-np.mean(ewArr)),bins=30,color=clr)
        axErr[i,4].axvline((e-np.mean(ewArr)),ls='dashed',c='black')
            
        axErr[i,5].hist((lumArr-np.mean(lumArr)),bins=30,color=clr)
        axErr[i,5].axvline((l-np.mean(lumArr)),ls='dashed',c='black')
        
        # Labeling columns
        if i==7:
            axErr[i,0].set_xlabel('(km s$^{-1}$)')
            axErr[i,1].set_xlabel('(km s$^{-1}$)')
            axErr[i,2].set_xlabel('(km s$^{-1}$)')
            axErr[i,3].set_xlabel('(km s$^{-1}$)')
            axErr[i,4].set_xlabel('(Å)')
            axErr[i,5].set_xlabel('(10$^{42}$ erg s$^{-1}$)')
        
        # Titling columns    
        if i==0:
            axErr[i,0].set_title('FWHM (Blue)')
            axErr[i,1].set_title('FWHM (Center)')
            axErr[i,2].set_title('FWHM (Red)')
            axErr[i,3].set_title('Peak Sep.')
            axErr[i,4].set_title('EW')
            axErr[i,5].set_title('Luminosity')
        
        # Labeling row with slit name
        axErr[i,5].yaxis.set_label_position("right")
        axErr[i,5].set_ylabel(slits[i])
        
        k=0
        
        for j in [fwhmBArr,fwhmCArr,fwhmRArr,psepArr,ewArr,lumArr]:
            
            at=AnchoredText('$\mu$='+str(round(np.mean(j),3))+'\n'+'$\sigma$='+str(round(np.std(j),3)),loc='upper left',frameon=False)
            
            axErr[i,k].add_artist(at)
            
            k=k+1
            
        figT,axT=plt.subplots(1,1)
        
        axT.plot(vArr,fArr*1e16,ds='steps-mid',c='black')
        axT.plot(vArr,gaussFit(vArr,*popt),ds='steps-mid',c='red',ls='dashed')
        axT.plot(vArr,fArr*1e16-gaussFit(vArr,*popt),ds='steps-mid',c='black',ls='dotted')
        
        axT.plot(hmxB, [halfB+cntm*1e16, halfB+cntm*1e16],c='black',ls='dashed')
        axT.plot(hmxC,[halfC+cntm*1e16,halfC+cntm*1e16],c='black',ls='dashed')
        axT.plot(hmxR, [halfR+cntm*1e16, halfR+cntm*1e16],c='black',ls='dashed')
        
        axT.axvline(b,ls='dashed',c='blue')
        axT.axvline(r,ls='dashed',c='red')
        
        axT.set_xlim(-1000.0,1000.0)
        axT.set_ylim(0.0)
        
        axT.set_title('Slit '+slits[i])
        
        axT.set_xlabel(vlabel)
        axT.set_ylabel(flabel)
        
        atT=AnchoredText('EW='+str(round(np.mean(ewArr),1))+'$\pm$'+str(round(np.std(ewArr),1))+'$\,$Å'+'\n'
                         +'Lum.='+str(round(np.mean(lumArr),1))+'$\pm$'+str(round(np.std(lumArr),1))+'$\,$10$^{42}$ erg s$^{-1}$'+'\n'
                         +'Peak Sep.='+str(round(np.mean(psepArr),1))+'$\pm$'+str(round(np.std(psepArr),1))+'$\,$km s$^{-1}$'+'\n'+'\n'
                         +'FWHM (red)='+str(round(np.mean(fwhmRArr),1))+'$\pm$'+str(round(np.std(fwhmRArr),1))+'$\,$km s$^{-1}$'+'\n'
                         +'FWHM (ctr)='+str(round(np.mean(fwhmCArr),1))+'$\pm$'+str(round(np.std(fwhmCArr),1))+'$\,$km s$^{-1}$'+'\n'
                         +'FWHM (blue)='+str(round(np.mean(fwhmBArr),1))+'$\pm$'+str(round(np.std(fwhmBArr),1))+'$\,$km s$^{-1}$'
                         ,loc='upper left',prop=dict(size=8),frameon=False)
        axT.add_artist(atT)
        
        figT.savefig(figs+'/'+slits[i]+'T.pdf',bbox_inches='tight')
    
    # If there is a center peak and a red peak, 
    # but no blue peak
    elif cPeak[i]==True and bPeak[i]==False:
        
        fitGuess=(ampGuess[i],ctrGuess[i],stdGuess[i])
        sampleRange=(Min[i],Max[i])
        popt,pcov=fitNormPeak(vArr,fArr*1e16,*sampleRange,params=fitGuess,bounds=([0.0,ctrLBnd[i],0.0],[np.inf,ctrUBnd[i],stdUBnd[i]]))
        
        #ctrR=centroid(vArr,fArr*1e16-gaussFit(vArr,*popt),popt[1],500.0)
        
        cntm=cont(wArr,fArr,contLBnd[i],contUBnd[i])
        
        e,f,l=ewfluxlum(wArr,fArr,cntm,z[i])
        l=l*1e-42
        
        deriv=np.gradient(smooth(fArr*1e16-gaussFit(vArr,*popt),5),vArr,edge_order=2)
        
        b,r,intp=peaks(vArr,deriv,popt[1],popt[2],slits[i])
        
        hmxR,halfR=half_max_x(vArr,fArr*1e16-gaussFit(vArr,*popt)-cntm*1e16,popt[1],750.0,r)
        hmxC,halfC=half_max_x(vArr,gaussFit(vArr,*popt)-cntm*1e16,-1000.0,1000.0,popt[1])
        
        n=0
        fwhmRArr=np.array([])
        fwhmCArr=np.array([])
        fwhmBArr=np.array([])
        psepArr=np.array([])
        ewArr=np.array([])
        lumArr=np.array([])
        
        while n!=1000:
    
            smthg=np.random.normal(fArr,nArr)
        
            cntmN=cont(wArr,smthg,contLBnd[i],contUBnd[i])
            
            eN,fN,lN=ewfluxlum(wArr,smthg,cntmN,z[i])
            lN=lN*1e-42
    
            popt1,pcov1=fitNormPeak(vArr,smthg*1e16,*sampleRange,params=popt,bounds=([0.0,ctrLBnd[i],0.0],[np.inf,ctrUBnd[i],stdUBnd[i]]))
    
            peak=gaussFit(vArr,*popt1)
            noCPeak=smthg*1e16-gaussFit(vArr,*popt1)
            
            derivN=np.gradient(smooth(smthg*1e16-gaussFit(vArr,*popt1),5),vArr,edge_order=2)
            
            bN,rN,intpN=peaks(vArr,derivN,popt1[1],popt1[2],slits[i])
        
            hmxRN,halfRN=half_max_x(vArr,noCPeak-cntmN*1e16,popt1[1],1000.0,rN)
            hmxCN,halfCN=half_max_x(vArr,peak-cntmN*1e16,-1000.0,1000.0,popt1[1])
            
            #print(hmxRN)

            #plt.plot(vArr,smthg*1e16-gaussFit(vArr,*popt1),c='blue',ds='steps-mid')
            #plt.plot(vArr,smthg*1e16,c='black',ds='steps-mid')
            #plt.plot(vArr,gaussFit(vArr,*popt1),ls='dashed',c='red',ds='steps-mid')
            #plt.plot(hmxRN,[halfRN+cntmN,halfRN+cntmN],c='black')
            #plt.plot(hmxCN,[halfCN+cntmN,halfCN+cntmN],c='black')
            #plt.axvline(ctrR,ls='-')
            #plt.xlim(-1000.0,1000.0)
            #plt.title(slits[i])
            #plt.show()
    
            #print(*popt1)
        
            if slits[i]=='X':
        
                f,a=plt.subplots(2,1,sharex=True)
            
                a[0].plot(vArr,derivN)
                a[0].axvline(bN,ls='dashed',c='blue',label=str(bN))
                a[0].axvline(rN,ls='dashed',c='red',label=str(rN))
                a[0].axhline(0.0,ls='dashed',c='black')
                
                a[0].legend(loc='upper right')
            
                a[1].plot(vArr,smthg*1e16-gaussFit(vArr,*popt1))
                a[1].axvline(bN,ls='dashed',c='blue')
                a[1].axvline(rN,ls='dashed',c='red')
                a[1].axvline(b,ls='dotted',c='blue')
                a[1].axvline(r,ls='dotted',c='red')
            
                a[1].set_xlim(-1000.0,1000.0)
            
                plt.show()
    
            fwhmRN=np.array([hmxRN[1]-hmxRN[0]])
            fwhmCN=np.array([hmxCN[1]-hmxCN[0]])
            
            fwhmRArr=np.append(fwhmRArr,fwhmRN)
            fwhmCArr=np.append(fwhmCArr,fwhmCN)
            
            ewArr=np.append(ewArr,eN)
            lumArr=np.append(lumArr,lN)
            
            n=n+1
            
        fwhmRArr=fwhmRArr[~np.isnan(fwhmRArr)]
        print(np.shape(fwhmRArr))

        axErr[i,1].hist((fwhmCArr-np.mean(fwhmCArr)),bins=30)
        axErr[i,1].axvline(((hmxC[1]-hmxC[0])-np.mean(fwhmCArr)),ls='dashed',c='black')
        if i==0:
            axErr[i,1].set_title('FWHM (Center)')
            
        axErr[i,2].hist((fwhmRArr-np.mean(fwhmRArr)),bins=30)
        axErr[i,2].axvline(((hmxR[1]-hmxR[0])-np.mean(fwhmRArr)),ls='dashed',c='black')
        if i==0:
            axErr[i,2].set_title('FWHM (Red)')
        
        axErr[i,4].hist((ewArr-np.mean(ewArr)),bins=30)
        axErr[i,4].axvline((e-np.mean(ewArr)),ls='dashed',c='black')
        if i==0:
            axErr[i,4].set_title('EW')
            
        axErr[i,5].hist((lumArr-np.mean(lumArr)),bins=30)
        axErr[i,5].axvline((l-np.mean(lumArr)),ls='dashed',c='black')
        if i==0:
            axErr[i,5].set_title('Luminosity')
        
        axErr[i,5].yaxis.set_label_position("right")
        axErr[i,5].set_ylabel(slits[i])
        
        k=1
        
        for j in [fwhmCArr,fwhmRArr,ewArr,lumArr]:
            
            at=AnchoredText('$\mu$='+str(round(np.mean(j),3))+'\n'+'$\sigma$='+str(round(np.std(j),3)),loc='upper left',frameon=False)
            
            axErr[i,k].add_artist(at)
            
            if k==2:
                
                k=k+1
            
            k=k+1

        figT,axT=plt.subplots(1,1)
        
        axT.plot(vArr,fArr*1e16,ds='steps-mid',c='black')
        axT.plot(vArr,gaussFit(vArr,*popt),ds='steps-mid',c='red',ls='dashed')
        axT.plot(vArr,fArr*1e16-gaussFit(vArr,*popt),ds='steps-mid',c='black',ls='dotted')
        
        axT.plot(hmxC,[halfC+cntm*1e16,halfC+cntm*1e16],c='black',ls='dashed')
        axT.plot(hmxR, [halfR+cntm*1e16, halfR+cntm*1e16],c='black',ls='dashed')
        
        axT.axvline(r,ls='dashed',c='red')
        
        axT.set_xlim(-1000.0,1000.0)
        axT.set_ylim(0.0)
        
        axT.set_title('Slit '+slits[i])
        
        axT.set_xlabel(vlabel)
        axT.set_ylabel(flabel)
        
        atT=AnchoredText('EW='+str(round(np.mean(ewArr),1))+'$\pm$'+str(round(np.std(ewArr),1))+'$\,$Å'+'\n'
                         +'Lum.='+str(round(np.mean(lumArr),1))+'$\pm$'+str(round(np.std(lumArr),1))+'$\,$10$^{42}$ erg s$^{-1}$'+'\n\n'
                         +'FWHM (red)='+str(round(np.mean(fwhmRArr),1))+'$\pm$'+str(round(np.std(fwhmRArr),1))+'$\,$km s$^{-1}$'+'\n'
                         +'FWHM (ctr)='+str(round(np.mean(fwhmCArr),1))+'$\pm$'+str(round(np.std(fwhmCArr),1))+'$\,$km s$^{-1}$'+'\n'
                         ,loc='upper left',prop=dict(size=8),frameon=False)
        axT.add_artist(atT)
        
        figT.savefig(figs+'/'+slits[i]+'T.pdf',bbox_inches='tight')
            
    # If there is only a red peak
    elif cPeak[i]==False and bPeak[i]==False:
        
        cntm=cont(wArr,fArr,contLBnd[i],contUBnd[i])
        
        e,f,l=ewfluxlum(wArr,fArr,cntm,z[i])
        l=l*1e-42
        
        deriv=np.gradient(smooth(fArr*1e16,5),vArr,edge_order=2)
        
        b,r,intp=peaks(vArr,deriv,popt1[1],popt1[2],slits[i])
        
        hmxR,halfR=half_max_x(vArr,fArr*1e16-cntm*1e16,0.0,750.0,r)
        
        #plt.close('all')
        
        n=0
        fwhmRArr=np.array([])
        ewArr=np.array([])
        lumArr=np.array([])
        
        while n!=1000:
    
            smthg=np.random.normal(fArr,nArr)
        
            cntmN=cont(wArr,smthg,contLBnd[i],contUBnd[i])
            
            eN,fN,lN=ewfluxlum(wArr,smthg,cntmN,z[i])
            lN=lN*1e-42
            
            derivN=np.gradient(smooth(fArr*1e16,5),vArr,edge_order=2)
            
            bN,rN,intpN=peaks(vArr,derivN,popt1[1],popt1[2],slits[i])
    
            hmxRN,halfRN=half_max_x(vArr,smthg*1e16-cntmN*1e16,0.0,750.0,rN)
    
            #plt.plot(vArr,smthg*1e16,c='black',ds='steps-mid')
            #plt.plot(vArr,nArr*1e16,c='black',ds='steps-mid')
            #plt.plot(hmxRN,[halfRN+cntmN,halfRN+cntmN],c='black')
            #plt.axvline(ctrR,ls='-')
            #plt.xlim(-1000.0,1000.0)
            #plt.title(slits[i])
            #plt.show()
    
            fwhmRN=hmxRN[1]-hmxRN[0]
            
            fwhmRArr=np.append(fwhmRArr,fwhmRN)
            
            ewArr=np.append(ewArr,eN)
            lumArr=np.append(lumArr,lN)
            
            n=n+1
        
        fwhmRArr=fwhmRArr[~np.isnan(fwhmRArr)]
        print(np.shape(fwhmRArr))
        
        axErr[i,2].hist((fwhmRArr-np.mean(fwhmRArr)),bins=30)
        axErr[i,2].axvline(((hmxR[1]-hmxR[0])-np.mean(fwhmRArr)),ls='dashed',c='black')
        
        axErr[i,4].hist((ewArr-np.mean(ewArr)),bins=30)
        axErr[i,4].axvline((e-np.mean(ewArr)),ls='dashed',c='black')
        if i==0:
            axErr[i,4].set_title('EW')
            
        axErr[i,5].hist((lumArr-np.mean(lumArr)),bins=30)
        axErr[i,5].axvline((l-np.mean(lumArr)),ls='dashed',c='black')
        if i==0:
            axErr[i,5].set_title('Luminosity')
        
        axErr[i,5].yaxis.set_label_position("right")
        axErr[i,5].set_ylabel(slits[i])
        
        k=2
        
        for j in [fwhmRArr,ewArr,lumArr]:
            
            at=AnchoredText('$\mu$='+str(round(np.mean(j),3))+'\n'+'$\sigma$='+str(round(np.std(j),3)),loc='upper left',frameon=False)
            
            axErr[i,k].add_artist(at)
            
            if k==2:
                
                k=k+1
            
            k=k+1
        
        figT,axT=plt.subplots(1,1)
        
        axT.plot(vArr,fArr*1e16,ds='steps-mid',c='black')
        
        axT.plot(hmxR, [halfR+cntm*1e16, halfR+cntm*1e16],c='black',ls='dashed')
        
        axT.axvline(r,ls='dashed',c='red')
        
        axT.set_xlim(-1000.0,1000.0)
        axT.set_ylim(0.0)
        
        axT.set_title('Slit '+slits[i])
        
        axT.set_xlabel(vlabel)
        axT.set_ylabel(flabel)
        
        atT=AnchoredText('EW='+str(round(np.mean(ewArr),1))+'$\pm$'+str(round(np.std(ewArr),1))+'$\,$Å'+'\n'
                         +'Lum.='+str(round(np.mean(lumArr),1))+'$\pm$'+str(round(np.std(lumArr),1))+'$\,$10$^{42}$ erg s$^{-1}$'+'\n\n'
                         +'FWHM (red)='+str(round(np.mean(fwhmRArr),1))+'$\pm$'+str(round(np.std(fwhmRArr),1))+'$\,$km s$^{-1}$'+'\n'
                         ,loc='upper left',prop=dict(size=8),frameon=False)
        axT.add_artist(atT)
        
        figT.savefig(figs+'/'+slits[i]+'T.pdf',bbox_inches='tight')
            
figErr.supxlabel('x-$\mu$')
figErr.supylabel('Count')
        
figErr.tight_layout()

figErr.savefig(figs+'/err.png',dpi=100,bbox_inches='tight')

plt.show()